# Module 6: RNNs, LSTMs & Sequential Data

---
## Why Do Sequences Need Special Treatment?

### 🧠 Brain Analogy
"The movie was NOT good" vs "The movie was NOT bad" — the word "not" flips the meaning completely. A regular neural network processes each word independently, missing that "not" changes what follows. Your brain reads sequentially with MEMORY — "not" stays in working memory and colours everything after it.

RNNs and LSTMs do the same: process word by word, carrying a "running understanding" (hidden state) that updates with each new word.

### ⚙️ Engineer Analogy
Regular networks assume inputs are independent — wrong for text, time series, audio. Sequential data needs: (1) memory, (2) selective updating, (3) variable-length handling. RNN/LSTM/GRU solve all three.
```
RNN  → one note (hidden state). Forgets early words (vanishing gradients).
LSTM → two memories: scratchpad (h) + diary (c). Diary keeps info for hundreds of steps.
GRU  → simplified LSTM, one combined memory. Fewer params, often same accuracy.
```

**Level:** Intermediate  
**Duration:** ~4 hours  
**Dataset:** IMDB Movie Reviews (via `datasets` library — same as [Kaggle IMDB sentiment](https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews))  
**Real-World Use Case:** Sentiment analysis, chatbots, time series forecasting, log anomaly detection

## What You'll Learn
- Text preprocessing: tokenization, vocabulary building, padding
- Embedding layers (`nn.Embedding`)
- Vanilla RNN, LSTM, and GRU — differences & when to use each
- Bidirectional LSTMs
- Packing padded sequences for efficiency
- Bonus: 1D CNN for text (faster alternative)

## The Problem with Regular Networks on Sequences
```
Review: "The movie was not good, but I liked it anyway"

Regular NN: processes each word independently → misses "not good" context

RNN/LSTM:   maintains a hidden state (memory) → understands "not" modifies "good"
```

## RNN vs LSTM vs GRU
```
RNN  → simple, fast, suffers from vanishing gradients on long sequences
LSTM → adds cell state + 3 gates (forget/input/output), handles long dependencies
GRU  → simplified LSTM with 2 gates, often matches LSTM with fewer parameters
```

In [ ]:
# 🧠 Tools for sequence processing — pack/pad for handling variable-length reviews efficiently
# ⚙️ pack_padded_sequence + pad_packed_sequence: LSTM skips padding tokens for cleaner gradients
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import re
import string

torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

## 6.1  Load IMDB Dataset

### 🧠 Brain Analogy
Before the model can read a review, translate it: "great movie!" → ["great", "movie"] → [542, 17] → embedding vectors. Each step makes the data more usable. Build the vocabulary from TRAINING data only — unseen test words map to UNK.

### ⚙️ Engineer Analogy
Vocabulary: bijective `word→int` mapping. MAX_VOCAB=20,000 covers ~95% of English. PAD=0 (placeholder), UNK=1 (out-of-vocabulary). Counter.most_common(N): efficient O(n log n) frequency ranking.

```bash
# Option A — Hugging Face datasets:
pip install datasets

# Option B — Kaggle:
kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
```

In [ ]:
# 🧠 Load IMDB movie reviews — 25k positive, 25k negative — from Hugging Face
# ⚙️ datasets library is the standard for loading NLP benchmarks without manual download
# ── Load from Hugging Face (no Kaggle account needed) ────────────────────
try:
    from datasets import load_dataset
    hf_data = load_dataset('imdb', split={'train': 'train', 'test': 'test'})
    train_texts  = hf_data['train']['text']
    train_labels = hf_data['train']['label']
    test_texts   = hf_data['test']['text']
    test_labels  = hf_data['test']['label']
    print("Loaded from Hugging Face datasets")
except ImportError:
    # Fallback: torchtext built-in IMDB
    import torchtext
    train_data, test_data = torchtext.datasets.IMDB(root='./data')
    train_labels, train_texts = zip(*list(train_data))
    test_labels,  test_texts  = zip(*list(test_data))
    train_labels = [1 if l == 'pos' else 0 for l in train_labels]
    test_labels  = [1 if l == 'pos' else 0 for l in test_labels]
    print("Loaded from torchtext")

print(f"Train: {len(train_texts)}, Test: {len(test_texts)}")
print(f"\nSample review (first 200 chars):\n{train_texts[0][:200]}...")
print(f"\nLabel: {train_labels[0]}  (1=positive, 0=negative)")

## 6.2  Text Preprocessing & Vocabulary

### 🧠 Brain Analogy
Encode: translate each review to word-ID numbers. Padding challenge: reviews have DIFFERENT lengths but batches need FIXED size. Solution: add 0s (blank tokens) to short reviews — like blank pages at the end of short chapters. `lengths` tensor remembers the TRUE length so the LSTM can stop processing at the right point.

### ⚙️ Engineer Analogy
`pad_sequence`: takes variable-length tensors → padded matrix shape (B, max_len). `lengths`: true pre-padding length per sequence — needed for `pack_padded_sequence` to skip PAD tokens during LSTM computation. `padding_value=0`: PAD → zero embedding → zero gradient contribution.



In [ ]:
# 🧠 Build a word-to-number dictionary from training reviews
# ⚙️ Counter.most_common: O(n log n) frequency counting; top 20k words covers 95% of text
# ── Tokenizer ────────────────────────────────────────────────────────────
def tokenize(text: str) -> list:
    """Simple whitespace tokenizer with basic cleaning."""
    text = text.lower()
    text = re.sub(r'<[^>]+>', ' ', text)              # remove HTML tags
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text.split()


# ── Build vocabulary from training data ─────────────────────────────────
MAX_VOCAB  = 20_000
MAX_SEQ_LEN = 256   # truncate/pad to this length

counter = Counter()
for text in train_texts:
    counter.update(tokenize(text))

# Special tokens
PAD_TOKEN, UNK_TOKEN = '<PAD>', '<UNK>'
vocab = {PAD_TOKEN: 0, UNK_TOKEN: 1}
vocab.update({
    word: idx + 2
    for idx, (word, _) in enumerate(counter.most_common(MAX_VOCAB - 2))
})
PAD_IDX = vocab[PAD_TOKEN]
UNK_IDX = vocab[UNK_TOKEN]

print(f"Vocabulary size: {len(vocab):,}")
print(f"Most common words: {list(counter.most_common(10))}")

In [ ]:
# 🧠 Translate reviews to ID sequences; pad short ones so all fit in the same batch
# ⚙️ collate_fn: called by DataLoader to combine variable-length sequences into fixed batches
def encode(text: str, vocab: dict, max_len: int) -> torch.Tensor:
    """Tokenize → numericalize → truncate/pad."""
    tokens = tokenize(text)[:max_len]
    ids    = [vocab.get(t, UNK_IDX) for t in tokens]
    return torch.tensor(ids, dtype=torch.long)


class IMDBDataset(Dataset):
    def __init__(self, texts, labels, vocab, max_len):
        self.encodings = [encode(t, vocab, max_len) for t in texts]
        self.labels    = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.encodings[idx], self.labels[idx]


def collate_fn(batch):
    """Pad sequences in a batch to equal length."""
    sequences, labels = zip(*batch)
    lengths  = torch.tensor([len(s) for s in sequences])
    padded   = pad_sequence(sequences, batch_first=True, padding_value=PAD_IDX)
    return padded, torch.stack(list(labels)), lengths


# Use 5k train + 2k test for speed (increase for better results)
N_TRAIN, N_TEST = 5000, 2000
train_ds = IMDBDataset(train_texts[:N_TRAIN], train_labels[:N_TRAIN], vocab, MAX_SEQ_LEN)
test_ds  = IMDBDataset(test_texts[:N_TEST],  test_labels[:N_TEST],  vocab, MAX_SEQ_LEN)

train_loader = DataLoader(train_ds, batch_size=64,  shuffle=True,
                          collate_fn=collate_fn)
test_loader  = DataLoader(test_ds,  batch_size=128, shuffle=False,
                          collate_fn=collate_fn)

# Inspect a batch
x_b, y_b, lens = next(iter(train_loader))
print(f"Batch X: {x_b.shape}, Y: {y_b.shape}, Lengths: {lens[:5].tolist()}")

## 6.3  Understanding Embeddings

### 🧠 Brain Analogy
An embedding gives each word a position in a "meaning space" where similar words are close together. "cat" and "dog" are nearby (both pets). "happy" and "joyful" are almost identical. The model LEARNS these positions during training — words appearing in similar contexts end up close in embedding space.

### ⚙️ Engineer Analogy
`nn.Embedding(vocab_size, embed_dim, padding_idx=0)` = (N, D) learnable lookup table. Row lookup is O(1). padding_idx=0: row 0 = all zeros, no gradient contribution. Initialise with pretrained GloVe/Word2Vec via `nn.Embedding.from_pretrained(vectors)` for better performance.



In [ ]:
# 🧠 Each word-ID → an 8-dimensional "meaning vector" in semantic space
# ⚙️ Embedding = learned lookup table (vocab_size × embed_dim). Padding idx=0 → zeros, no gradient.
# nn.Embedding: lookup table of shape (vocab_size, embedding_dim)
# Maps integer token IDs → dense vectors

emb = nn.Embedding(num_embeddings=100, embedding_dim=8, padding_idx=0)
token_ids = torch.tensor([[1, 5, 3, 0, 0],    # batch item 1 (padded at end)
                           [2, 7, 4, 6, 9]])   # batch item 2
embedded = emb(token_ids)
print(f"Input shape:  {token_ids.shape}")
print(f"Output shape: {embedded.shape}")
print("Each token mapped to 8-dim vector")
print(f"Padding idx (0) embedding: {emb(torch.tensor([0]))}")

## 6.4  Model Architectures

### 🧠 Brain Analogy
**LSTM**: two memories. "The movie was, despite the terrible reviews, actually quite good." By "good" you need: short-term ("actually quite") AND long-term ("despite terrible reviews"). LSTM's cell state (c) = the diary that holds long-range context. Three gates: forget (erase stale info), input (remember important new words), output (what to use right now).

**GRU**: combines forget+input into one update gate. Simpler, fewer params, good default.

**Bidirectional**: reads sentence forwards AND backwards — gets context from both directions for each word.

**TextCNN**: scans ALL windows of 3/4/5 words simultaneously. Very fast (fully parallel) but limited long-range context.

### ⚙️ Engineer Analogy
LSTM cell: `c_t = f_t ⊙ c_{t-1} + i_t ⊙ g_t`. Additive update = direct gradient highway → no vanishing. `pack_padded_sequence` = skip PAD tokens → cleaner gradients. `clip_grad_norm_(1.0)` = cap total gradient norm to prevent BPTT explosion.



In [ ]:
# 🧠 Three reading strategies: word-by-word-with-memory (LSTM/GRU) vs parallel-scanner (TextCNN)
# ⚙️ Bidirectional LSTM: concat hidden[-2] (forward) and hidden[-1] (backward) for full context
# ── 1. Simple LSTM Sentiment Classifier ───────────────────────────────────
class LSTMClassifier(nn.Module):
    """Bidirectional LSTM for text classification."""

    def __init__(self, vocab_size, embed_dim=128, hidden_dim=256,
                 n_layers=2, bidirectional=True, dropout=0.5, pad_idx=0):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embed_dim,
                                       padding_idx=pad_idx)
        self.lstm = nn.LSTM(
            input_size=embed_dim,
            hidden_size=hidden_dim,
            num_layers=n_layers,
            batch_first=True,
            dropout=dropout if n_layers > 1 else 0,
            bidirectional=bidirectional
        )
        self.dropout = nn.Dropout(dropout)

        # Bidirectional → 2× hidden_dim
        in_dim = hidden_dim * 2 if bidirectional else hidden_dim
        self.classifier = nn.Sequential(
            nn.Linear(in_dim, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, 1)   # binary output
        )

    def forward(self, x, lengths):
        # x: (batch, seq_len), lengths: actual lengths before padding

        embedded = self.dropout(self.embedding(x))  # (B, L, embed_dim)

        # Pack padded sequences for efficiency
        packed = pack_padded_sequence(embedded, lengths.cpu(),
                                       batch_first=True, enforce_sorted=False)
        packed_output, (hidden, _) = self.lstm(packed)

        # Take last hidden state from both directions
        # hidden: (n_layers * n_directions, B, hidden_dim)
        hidden = self.dropout(torch.cat(
            [hidden[-2], hidden[-1]], dim=1   # last layer, both directions
        ))   # (B, hidden_dim*2)

        return self.classifier(hidden).squeeze(1)   # (B,)


# ── 2. GRU variant ───────────────────────────────────────────────────────
class GRUClassifier(nn.Module):
    """GRU-based classifier — fewer params than LSTM, often similar performance."""

    def __init__(self, vocab_size, embed_dim=128, hidden_dim=256,
                 n_layers=2, bidirectional=True, dropout=0.5, pad_idx=0):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
        self.gru = nn.GRU(
            embed_dim, hidden_dim, n_layers,
            batch_first=True,
            dropout=dropout if n_layers > 1 else 0,
            bidirectional=bidirectional
        )
        self.dropout = nn.Dropout(dropout)
        in_dim = hidden_dim * 2 if bidirectional else hidden_dim
        self.fc = nn.Linear(in_dim, 1)

    def forward(self, x, lengths):
        emb = self.dropout(self.embedding(x))
        packed = pack_padded_sequence(emb, lengths.cpu(),
                                       batch_first=True, enforce_sorted=False)
        _, hidden = self.gru(packed)
        out = torch.cat([hidden[-2], hidden[-1]], dim=1)
        return self.fc(self.dropout(out)).squeeze(1)


# ── 3. 1D CNN (fast alternative to RNN) ──────────────────────────────────
class TextCNN(nn.Module):
    """Text CNN with multiple kernel sizes (Kim 2014)."""

    def __init__(self, vocab_size, embed_dim=128, n_filters=100,
                 filter_sizes=(3, 4, 5), dropout=0.5, pad_idx=0):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
        self.convs = nn.ModuleList([
            nn.Conv1d(embed_dim, n_filters, fs) for fs in filter_sizes
        ])
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(len(filter_sizes) * n_filters, 1)

    def forward(self, x, lengths=None):
        emb = self.embedding(x).permute(0, 2, 1)   # (B, embed, L)
        pooled = [torch.relu(conv(emb)).max(dim=2)[0] for conv in self.convs]
        cat = self.dropout(torch.cat(pooled, dim=1))
        return self.fc(cat).squeeze(1)


VOCAB_SIZE = len(vocab)

lstm_model = LSTMClassifier(VOCAB_SIZE, pad_idx=PAD_IDX).to(device)
gru_model  = GRUClassifier(VOCAB_SIZE,  pad_idx=PAD_IDX).to(device)
cnn_model  = TextCNN(VOCAB_SIZE,        pad_idx=PAD_IDX).to(device)

for name, m in [('LSTM', lstm_model), ('GRU', gru_model), ('TextCNN', cnn_model)]:
    params = sum(p.numel() for p in m.parameters())
    print(f"{name}: {params:,} parameters")

## 6.5  Training Loop

### 🧠 Brain Analogy
Same 5-step training loop, plus gradient clipping — a circuit breaker for the error signal. Without it, errors travelling back through many time steps can explode exponentially. `clip_grad_norm_(1.0)` = "if the error signal gets too strong, cap it at 1."

### ⚙️ Engineer Analogy
`clip_grad_norm_(params, 1.0)`: clips TOTAL gradient norm to 1.0. Critical for BPTT — without it gradients grow exponentially through long sequences. Standard practice for ALL RNN training. Never skip this.



In [ ]:
# 🧠 Train all three models on same reviews — who learns sentiment best?
# ⚙️ clip_grad_norm_(1.0): CRITICAL for RNNs — prevents exploding gradients through long sequences
def train_text_model(model, train_loader, test_loader, epochs=10, lr=1e-3, name='model'):
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2)

    history = {'tr_acc': [], 'vl_acc': []}
    best_acc = 0.0

    for epoch in range(1, epochs + 1):
        # Train
        model.train()
        correct, total = 0, 0
        for X, y, lens in train_loader:
            X, y, lens = X.to(device), y.to(device), lens
            optimizer.zero_grad()
            logits = model(X, lens)
            loss   = criterion(logits, y)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # prevent exploding gradients
            optimizer.step()
            preds   = (torch.sigmoid(logits) >= 0.5).long()
            correct += (preds == y.long()).sum().item()
            total   += len(y)
        tr_acc = correct / total

        # Eval
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for X, y, lens in test_loader:
                X, y = X.to(device), y.to(device)
                preds = (torch.sigmoid(model(X, lens)) >= 0.5).long()
                correct += (preds == y.long()).sum().item()
                total   += len(y)
        vl_acc = correct / total
        scheduler.step(1 - vl_acc)

        history['tr_acc'].append(tr_acc)
        history['vl_acc'].append(vl_acc)

        if vl_acc > best_acc:
            best_acc = vl_acc
            torch.save(model.state_dict(), f'{name}_best.pth')

        print(f"[{name}] Epoch {epoch:2d}/{epochs} | "
              f"train {tr_acc:.3f} | test {vl_acc:.3f}")

    return history, best_acc


# Train all three
results = {}
for name, model in [('LSTM', lstm_model), ('GRU', gru_model), ('TextCNN', cnn_model)]:
    print(f"\n{'='*50}")
    print(f"Training {name}")
    history, best = train_text_model(model, train_loader, test_loader,
                                      epochs=10, name=name)
    results[name] = {'history': history, 'best': best}

## 6.6  Compare Models

### 🧠 Brain Analogy
Three different reading strategies, one test. LSTM/GRU handle "not good" and "actually quite good despite terrible reviews" better (long-range context). TextCNN is fastest. All three achieve ~85-90% on IMDB. For state-of-the-art: BERT/GPT (transformers) — that's the next frontier.

### ⚙️ Engineer Analogy
TextCNN: fully parallelisable → fast GPU training. LSTM/GRU: sequential time dependency → can't parallelise across sequence. Transformers: parallel attention over all positions → best accuracy + parallelism, but need more data/memory.



In [ ]:
# 🧠 Side-by-side: sequential reader (LSTM/GRU) vs parallel scanner (TextCNN) — who wins?
# ⚙️ Solid line = test accuracy; dashed = train; bar chart shows best test accuracy per model
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 4))
colors = {'LSTM': 'blue', 'GRU': 'orange', 'TextCNN': 'green'}

for name, data in results.items():
    ax1.plot(data['history']['tr_acc'], '--', color=colors[name], alpha=0.5)
    ax1.plot(data['history']['vl_acc'],  '-', color=colors[name], label=name)

ax1.set_title('Test Accuracy (solid) vs Train (dashed)')
ax1.set_xlabel('Epoch'); ax1.set_ylabel('Accuracy')
ax1.legend()

best_accs = {k: v['best'] for k, v in results.items()}
ax2.bar(best_accs.keys(), best_accs.values(),
        color=[colors[k] for k in best_accs])
for i, (k, v) in enumerate(best_accs.items()):
    ax2.text(i, v + 0.005, f'{v:.1%}', ha='center', fontweight='bold')
ax2.set_ylim(0, 1.05)
ax2.set_title('Best Test Accuracy')
ax2.set_ylabel('Accuracy')

plt.suptitle('IMDB Sentiment Analysis — Model Comparison')
plt.tight_layout()
plt.show()

## 6.7  Inference — Predict Sentiment

### 🧠 Brain Analogy
The trained LSTM reads a new review word by word, building a sentiment signal. By the end: "> 0.5 = positive, < 0.5 = negative." Confidence = how far from 0.5. "0.95 = very sure positive." "0.52 = barely positive, model is uncertain."

### ⚙️ Engineer Analogy
Inference: tokenize → encode → `unsqueeze(0)` (add batch dim) → `model.eval()` forward → `sigmoid(logit)` → threshold at 0.5. `torch.no_grad()` = no gradient tracking needed → ~30% faster.



In [ ]:
# 🧠 Feed new unseen reviews to the trained brain — what does it think about each one?
# ⚙️ unsqueeze(0) adds batch dim; sigmoid converts logit to probability [0,1]
# Load best LSTM model
lstm_model.load_state_dict(torch.load('LSTM_best.pth', map_location=device))
lstm_model.eval()

def predict_sentiment(text: str, model, vocab: dict, max_len: int) -> dict:
    tokens = encode(text, vocab, max_len).unsqueeze(0).to(device)  # (1, L)
    length = torch.tensor([tokens.shape[1]])

    with torch.no_grad():
        logit = model(tokens, length)
        prob  = torch.sigmoid(logit).item()

    return {
        'text': text[:80] + '...',
        'probability': prob,
        'sentiment': 'POSITIVE' if prob >= 0.5 else 'NEGATIVE',
        'confidence': max(prob, 1 - prob)
    }


test_reviews = [
    "Absolutely brilliant film! The acting was outstanding and the story was gripping.",
    "Terrible waste of time. The plot made no sense and the acting was wooden.",
    "It started slow but picked up midway. Not bad overall, decent entertainment.",
    "This movie changed my life. A masterpiece of modern cinema.",
    "I walked out after 30 minutes. Unwatchable garbage."
]

for review in test_reviews:
    result = predict_sentiment(review, lstm_model, vocab, MAX_SEQ_LEN)
    icon = '😊' if result['sentiment'] == 'POSITIVE' else '😞'
    print(f"{icon} [{result['sentiment']:8s} {result['confidence']:.0%}] "
          f"{result['text']}")

## 6.8  Understanding RNN Internals

### 🧠 Brain Analogy
Watch the LSTM's memory change word by word as it reads a sentence. Hidden units activate differently for "terribly" vs "redeemed". The sentiment signal shifts after "but" — the model re-evaluates. Like watching your reading comprehension unfold in slow motion.

### ⚙️ Engineer Analogy
`register_forward_hook` captures packed LSTM output. `pad_packed_sequence` unpacks to (seq_len, hidden*2). Heatmap: each row = one hidden unit, each column = one word position. Mechanistic interpretability — understanding what internal representations were learned.



In [ ]:
# 🧠 Watch the brain's memory change word-by-word — see sentiment shift after "but"
# ⚙️ Hook captures packed output → unpack → slice first 32 hidden units → heatmap
# ── Anatomy of LSTM ───────────────────────────────────────────────────────
lstm = nn.LSTM(input_size=10, hidden_size=20, num_layers=2,
               batch_first=True, bidirectional=False)

# Input: (batch, seq_len, input_size)
x = torch.randn(4, 15, 10)   # 4 sequences, length 15, 10 features
output, (hidden, cell) = lstm(x)

print("LSTM shapes:")
print(f"  Input          : {x.shape}")
print(f"  Output         : {output.shape}  ← all timestep outputs")
print(f"  Hidden (h_n)   : {hidden.shape}  ← final hidden state (n_layers, B, H)")
print(f"  Cell   (c_n)   : {cell.shape}    ← final cell state")
print()

# ── GRU ──────────────────────────────────────────────────────────────────
gru = nn.GRU(input_size=10, hidden_size=20, num_layers=2, batch_first=True)
output_gru, hidden_gru = gru(x)
print("GRU shapes:")
print(f"  Output : {output_gru.shape}")
print(f"  Hidden : {hidden_gru.shape}  ← GRU has no cell state (simpler!)")

In [ ]:
# 🧠 Peek at raw LSTM machinery — what shapes come out and what does each represent?
# ⚙️ output=(B,L,H*dirs), h_n=(layers*dirs,B,H), c_n=same. GRU has no cell state (simpler!).
# ── Visualise hidden state evolution across tokens ─────────────────────
lstm_model.eval()
review = "The movie started terribly but the ending completely redeemed it"
tokens = encode(review, vocab, MAX_SEQ_LEN).unsqueeze(0).to(device)
length = torch.tensor([tokens.shape[1]])

# Attach hook to capture hidden states at each timestep
all_outputs = []
def hook_fn(module, inp, out):
    packed_out, _ = out
    all_outputs.append(packed_out)

h = lstm_model.lstm.register_forward_hook(hook_fn)
with torch.no_grad():
    _ = lstm_model(tokens, length)
h.remove()

# Unpack outputs to get per-token hidden states
packed_out = all_outputs[0]
unpacked, _ = pad_packed_sequence(packed_out, batch_first=True)
hidden_states = unpacked[0].cpu().numpy()   # (seq_len, hidden*2)

word_tokens = tokenize(review)[:MAX_SEQ_LEN]

plt.figure(figsize=(14, 4))
plt.imshow(hidden_states[:len(word_tokens), :32].T, aspect='auto',
           cmap='RdBu', vmin=-1, vmax=1)
plt.xticks(range(len(word_tokens)), word_tokens, rotation=45, ha='right')
plt.ylabel('Hidden unit')
plt.title('LSTM Hidden State Activations per Token (first 32 units)')
plt.colorbar(label='Activation')
plt.tight_layout()
plt.show()

## 6.9  Architecture Comparison Table

### 🧠 Brain Analogy
Summary of all sequential reading strategies: RNN (simple, forgets), LSTM (full memory), GRU (simplified memory), TextCNN (fast parallel scanner), Transformer (state-of-the-art attention). Each tool has its use case.

### ⚙️ Engineer Analogy
`clip_grad_norm_` is ALWAYS needed for RNNs. `pack_padded_sequence` = standard efficiency for variable-length sequences. Transformers replaced RNNs for most NLP but RNNs still shine for real-time streaming data.

| Model | Pros | Cons | Best For |
|-------|------|------|----------|
| RNN | Simple, fast | Vanishing gradients on long seq | Very short sequences |
| LSTM | Long-range dependencies | Slower, more params than GRU | Long text, time series |
| GRU | Faster than LSTM, fewer params | Slightly worse on very long seq | General purpose sequences |
| TextCNN | Very fast, parallelizable | Limited long-range context | Short texts, high throughput |
| Transformer | Best accuracy, parallelizable | Memory O(n²), needs lots of data | State-of-the-art NLP |

## Sequential Data Cheatsheet

```python
# LSTM
nn.LSTM(input_size, hidden_size, num_layers, batch_first=True,
         bidirectional=True, dropout=0.5)

# GRU
nn.GRU(input_size, hidden_size, num_layers, batch_first=True)

# Pack padded (efficiency when sequences have different lengths)
packed = pack_padded_sequence(embedded, lengths, batch_first=True,
                               enforce_sorted=False)
output, (h_n, c_n) = lstm(packed)
output, lengths = pad_packed_sequence(output, batch_first=True)

# Gradient clipping (CRITICAL for RNNs — prevents exploding gradients)
nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
```

## Exercises

1. Download the [Kaggle IMDB dataset](https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews) and train on all 50k reviews.
2. Add **pre-trained GloVe embeddings** (`nn.Embedding.from_pretrained(glove_vectors)`) instead of random init.
3. Apply this same pipeline to **time series prediction** — replace text tokens with stock price windows.
4. Upgrade to a simple **Transformer encoder** using `nn.TransformerEncoder`. Compare accuracy.

---
## Congratulations! You've completed the PyTorch Learning Path!

### Next Steps
- Explore **Hugging Face Transformers** for state-of-the-art NLP
- Learn **PyTorch Lightning** for clean, scalable training code
- Try **Weights & Biases (wandb)** for experiment tracking
- Study **model deployment** with TorchServe or FastAPI
- Enter a [Kaggle competition](https://www.kaggle.com/competitions) to apply your skills!